# Documentation

## Objective(s)
1. Ingest event log file data file to extract key meta data

    
## Change Log
1. Lorem ipsum
    


# Initialisation

### Adjust Display Theme of Jupyter Notebook
Optional Step

Key Hotkeys:
1. Run cell: ctrl + enter
2. Delete Cell: DD
3. Undo Delete Cell: Z
4. Cut Cell: X
5. Copy Cell: C
6. Insert Cell Above: A
7. Insert Cell Below: B
8. Convert Cell to Code: Y
9. Convert Cell to Markdown: M

In [1]:
# install jupyterthemes
#!pip install jupyterthemes

# upgrade to latest version
#!pip install --upgrade jupyterthemes

In [2]:
#import jupyterthemes

In [3]:
# Adjust to Dark Theme
#jt -t chesterish

## Load Libraries

In [4]:
#pip install --user --upgrade pandas


In [5]:
# Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install schedule

In [6]:
# Import libraries
#import modin.pandas as pd
import pandas as pd
import numpy as np
import os
import pathlib
import datetime as dt
import time
import os
#import re

## Set Options

In [7]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

## Set Up Core Directories

In [8]:
# Check current directory location
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [9]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs')

# Check current directory location
cwd = os.getcwd()

# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [10]:
# Create Directory for Output Files Generated
if not os.path.exists('dataCleaned'):
    os.makedirs('dataCleaned')
    
# Inspect files in directory
fileList = os.listdir()
fileList

['associationRules',
 'dataCleaned',
 'desktop.ini',
 'dummyLog',
 'Original Sample from 27 Oct 2020 (simplified)',
 'Repair Logs',
 'Sample from 27 Oct 2020 (OG).zip',
 'sample_data_ats',
 'sample_data_ats.zip',
 'sample_data_cms',
 'sample_data_cms.zip',
 'sample_data_ecs',
 'sample_data_ecs.zip',
 'sample_data_HISevent',
 'taggedOutput',
 'testLog']

In [11]:
# Location of Alarm and Normal Event Files
alarmLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\AlarmLogs'
eventLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs'
testLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\TestLogs2'

# Set variables
# Alarm Events: 'AL'
# All Events: 'AE'

prefix = 'AE' 
batch = 'B001-'
serverEnv = 'CMS-'

## Access Files to be Processed

In [12]:
# Define root file directory folder where the files are being stored
if prefix == "AL":
    os.chdir(cwd + alarmLoc)
    #os.chdir(cwd + eventLoc)
elif prefix == "AE":
    os.chdir(cwd + eventLoc)
else:
    os.chdir(cwd + testLoc)
    
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs'

In [13]:
# Get the list of all files in directory tree at given path
fileList = list()
for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    fileList += [os.path.join(dirpath, file) for file in filenames] # use this if you want to append full URL
    #fileList += filenames
    
# Inspect data
len(fileList)

3353925

In [14]:
testFile = fileList[0]
testFile

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs\\0000001\\EventListTest_1611428800_253843'

In [15]:
# Get File Name without Path
os.path.basename(testFile)

'EventListTest_1611428800_253843'

In [16]:
# Get File Size in Bytes
os.path.getsize(testFile)

102

# Process Log Files

In [17]:
# Core Function for Cleaning (dependent on above functions)
# Excludes file saving
def cleaningScript_vector(fileInput, prefixInput = "Test"):
    
    ##################################################################
    # Import Dependant Libraries
    ##################################################################
    
    import sys, os
    if 'pd' not in sys.modules:
        import pandas as pd
    if 'np' not in sys.modules:
        import numpy as np
    if 'dt' not in sys.modules:
        import datetime as dt
    #if 'os' not in sys.modules:
    #    import os as os
    
    ##################################################################
    # Initialise Dependant Function 1
    ##################################################################
     
    def parseMetaData(f_meta):

        # Pre-process f_meta2
        DATETIME_RECEIVED = f_meta.split('End Notification: ')[1]

        return DATETIME_RECEIVED
     
    ##################################################################
    # Ingest File
    ##################################################################
    
    # Define File Name
    fileName = fileInput
    
    # Read in file
    file = open(fileName, 'r')
        
    # Convert file contents to a list
    fileContents = list(file)

    # Close file
    file.close()

               
    ##################################################################
    # Extract Key Components in Ingested File
    ##################################################################
    
    # Extract row containing meta data of file received datetime
    f_meta = fileContents[-1]

    ##################################################################
    # Process Meta Data
    ##################################################################
    
    # Insert File Name
    FileName = os.path.basename(fileInput)
    
    # Insert File Size In Bytes
    FileSize = os.path.getsize(fileInput)
    
    # Insert Time Received
    DATETIME_RECEIVED = parseMetaData(f_meta)
    
    df = pd.DataFrame.from_dict({
    'FileName': [FileName],
    'FileSize_Bytes': [FileSize],
    'DATETIME_RECEIVED': [DATETIME_RECEIVED]
    })
    
    
    # Delete redundant variables
    del f_meta, fileContents

    ##################################################################
    # Format Data Type & Perform Final Clean Up
    ################################################################## 

    # Preliminary Formatting
    df = df.convert_dtypes()

    # Format String Values to Datetime Format
    df['DATETIME_RECEIVED'] = pd.to_datetime(df['DATETIME_RECEIVED'], format='%m/%d/%y %H:%M:%S.%f', errors='coerce')
            
    return df

In [18]:
# Start clock to measure time taken
startTime = time.time()

testDF=cleaningScript_vector(testFile, prefix)

# End clock to measure time taken
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

Execution time in seconds: 0.013000011444091797
Execution time in minutes: 0.00021666685740152994
Execution time in hours: 3.611114290025499e-06


In [19]:
testDF.head()

,FileName,FileSize_Bytes,DATETIME_RECEIVED
0,EventListTest_1611428800_253843,102,2021-01-24 03:06:40.253


In [20]:
print(testDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           1 non-null      string        
 1   FileSize_Bytes     1 non-null      Int64         
 2   DATETIME_RECEIVED  1 non-null      datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 153.0 bytes
None


In [21]:
testFile2 = fileList[1]
testFile2

# Start clock to measure time taken
startTime = time.time()

testDF=cleaningScript_vector(testFile2, prefix)

# End clock to measure time taken
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

Execution time in seconds: 0.015000104904174805
Execution time in minutes: 0.0002500017484029134
Execution time in hours: 4.166695806715223e-06


In [22]:
testFile2

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs\\0000001\\EventListTest_1611428801_252605'

In [23]:
testFile

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs\\0000001\\EventListTest_1611428800_253843'

In [24]:
testDF.head()

,FileName,FileSize_Bytes,DATETIME_RECEIVED
0,EventListTest_1611428801_252605,29045,2021-01-24 03:06:41.252


In [25]:
print(testDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           1 non-null      string        
 1   FileSize_Bytes     1 non-null      Int64         
 2   DATETIME_RECEIVED  1 non-null      datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 153.0 bytes
None


## Design Loop to Process Data En Masse

**Core Functions**
1. Generate List of files in directory (DONE)
2. Loop through in file to process it
3. Append each process file to each other

**Advanced Functions**
1. Save appended files in after mulitples of N files have been appended (side step memory issues)
2. Maintain 2 files at all times


In [26]:
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs'

### Method 1: Enhance Looping
Loops through subdirectory sequentially and exports 1 file per subfolder - prevents memory ceiling error.

In [27]:
# Test Function (Vectorisation of Individual File Processing + DF concat + multiprocess)
import multiprocess as mp
#from multiprocessing import Pool # Does not work with iPython

# Script to generate list of all files within a particular folder
def list_files(dir):
    fileList = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            fileList.append(os.path.join(root, name))
    return fileList

# Start timer
startTime = time.time()
print("Start Time: " + time.ctime(startTime))

# Loop through each subfolder
# Process each file within subdirectory and export as a single CSV file
# Repeat for files in each subfolder
i = 0
folderList = os.listdir(os.getcwd())

# Batch process files by subfolder
for subfolder in folderList:
    subfolderDirectory = os.getcwd() + "\\" + subfolder
    fileList = list_files(subfolderDirectory)
    print(subfolderDirectory)
    print(len(fileList))
    #print(fileList[0])
    
    # Multiprocess files within subfolder
    if __name__ == '__main__':
        #prefixInput = multiprocessing.Manager.value(prefix)
        pool = mp.Pool(processes = (mp.cpu_count() - 1))
        results = pool.map(cleaningScript_vector, fileList)
        pool.close()
        pool.join()
        
        # Merge processed data
        df = pd.concat(results)
        
        # Export processed data within subfolder as a CSV file
        df.to_csv('../dataCleaned/' + serverEnv + prefix + '-FileMeta-' + batch + subfolder + '.csv', index=False)
    

    # Print Current Time
    print("Batch completed on: " + time.ctime(time.time()))
    
    # Print results of each run (subfolder)
    print(df.info())
    
    # Update counter
    i = i + 1

# Stop Timer
endTime = time.time()
executionTime = (endTime - startTime)

# Print Timing
print("End Time: " + time.ctime(endTime))
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

Start Time: Mon Jun 14 13:51:26 2021
C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000001
50000
Batch completed on: Mon Jun 14 13:53:12 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           50000 non-null  string        
 1   FileSize_Bytes     50000 non-null  Int64         
 2   DATETIME_RECEIVED  50000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 1.6 MB
None
C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000002
50000
Batch completed on: Mon Jun 14 13:55:01 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null 

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000014
50000
Batch completed on: Mon Jun 14 14:17:03 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           50000 non-null  string        
 1   FileSize_Bytes     50000 non-null  Int64         
 2   DATETIME_RECEIVED  50000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 1.6 MB
None
C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000015
50000
Batch completed on: Mon Jun 14 14:18:53 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------    

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000027
50000
Batch completed on: Mon Jun 14 14:41:07 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           50000 non-null  string        
 1   FileSize_Bytes     50000 non-null  Int64         
 2   DATETIME_RECEIVED  50000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 1.6 MB
None
C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000028
50000
Batch completed on: Mon Jun 14 14:42:58 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------    

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000040
50000
Batch completed on: Mon Jun 14 15:05:25 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           50000 non-null  string        
 1   FileSize_Bytes     50000 non-null  Int64         
 2   DATETIME_RECEIVED  50000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 1.6 MB
None
C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000041
50000
Batch completed on: Mon Jun 14 15:07:17 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------    

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000053
50000
Batch completed on: Mon Jun 14 15:29:46 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           50000 non-null  string        
 1   FileSize_Bytes     50000 non-null  Int64         
 2   DATETIME_RECEIVED  50000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 1.6 MB
None
C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000054
50000
Batch completed on: Mon Jun 14 15:31:38 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------    

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000066
50000
Batch completed on: Mon Jun 14 15:54:12 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FileName           50000 non-null  string        
 1   FileSize_Bytes     50000 non-null  Int64         
 2   DATETIME_RECEIVED  50000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), string(1)
memory usage: 1.6 MB
None
C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\sample_data_cms\batch 001 - 20201230 to 20210201\EventLogs\0000067
50000
Batch completed on: Mon Jun 14 15:56:04 2021
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------    

In [28]:
# Ring Beeper When Complete
import winsound
import time

duration1 = 400  # milliseconds
freq1 = 400  # Hz
duration2 = 600  # milliseconds
freq2 = 300  # Hz
repeatCount = 7
for n in range(repeatCount):
    winsound.Beep(freq1, duration1)
    winsound.Beep(freq2, duration2)
    time.sleep(1)

In [ ]:
# Shutdown computer after script runs (useful if the script takes really long to run)
import os
shutdownCountdown = 300 # time in seconds (wait for 5 min)
os.system("shutdown /s /t " + str(shutdownCountdown)) 

### Method 2: Brute Force for All Files within Folder
May result in memory errors if the number of files is too huge.

In [ ]:
# Check directory location
os.getcwd()

In [ ]:
# Get the list of all files in directory tree at given path
fileList = list()
for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    fileList += [os.path.join(dirpath, file) for file in filenames] # use this if you want to append full URL
    #fileList += filenames
    
# Inspect data
print(len(fileList))
print(fileList[1])

In [ ]:
# Test Function (Vectorisation of Individual File Processing + DF concat + multiprocess)
import multiprocess as mp
#from multiprocessing import Pool # Does not work with iPython
 
startTime = time.time()
if __name__ == '__main__':
    #prefixInput = multiprocessing.Manager.value(prefix)
    pool = mp.Pool(processes = (mp.cpu_count() - 1))
    results = pool.map(cleaningScript_vector, fileList)
    pool.close()
    pool.join()

    results_df = pd.concat(results)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

df.info()

In [ ]:
df.head()

In [ ]:
# Save File
df.to_csv('../dataCleaned/' + prefix + '-cleaned-' + str(indexCounter).zfill(3) + '.csv', index=False)

# Inspect files in saved directory
os.listdir(os.path.dirname(os.getcwd()) + '\\dataCleaned')